In [3]:
import csv
import tweepy
from dotenv import dotenv_values
import pandas as pd

In [15]:
#pip install tweepy

In [16]:
config = dotenv_values(".env")

In [17]:
accessToken = config["ACCESS_TOKEN"]
accessTokenSecret = config["ACCESS_TOKEN_SECRET"]
apiKey = config["API_KEY"]
apiSecret = config["API_KEY_SECRET"]
bearerToken = config["BEARER_TOKEN"]

In [19]:
#apiKey

In [18]:
# Authenticate to Twitter
auth = tweepy.OAuthHandler(apiKey, apiSecret) #API Keys
auth.set_access_token(accessToken, accessTokenSecret) #Access tokens

In [24]:
# Create API object
api = tweepy.API(auth, wait_on_rate_limit=True)

In [25]:
# Define the search query and date range
query = "Ronnie O'Sullivan OR #RonnieOSullivan -filter:retweets" #Search his name or hashtag of it
startDate = "2022-01-01"
endDate = "2022-12-31"

In [26]:
# Iterate through pages of results and save to CSV file
csvFile = open("ronnieTweets.csv", "w") #Name of file and write mode
csvWriter = csv.writer(csvFile)

for page in tweepy.Cursor(api.search_tweets, q=query, lang="en", since_id=startDate, until=endDate).pages():
    for tweet in page:
        csvWriter.writerow([tweet.id, tweet.created_at, tweet.text])
        
csvFile.close()

Forbidden: 403 Forbidden
453 - You currently have Essential access which includes access to Twitter API v2 endpoints only. If you need access to this endpoint, you’ll need to apply for Elevated access via the Developer Portal. You can learn more here: https://developer.twitter.com/en/docs/twitter-api/getting-started/about-twitter-api#v2-access-leve

Let this in to show the futility of it all. Twitter v2 endpoints only allow someone to search for recent tweets. Unsuitable for this project

In [28]:
import requests
import json
from twitter_stream import FilteredStream

In [33]:
search_url = "https://api.twitter.com/2/tweets/search/recent"

query_params = {
    #Escaping the character \\"
    "query" : "\"Ronnie O'Sullivan\" -is:retweet",
    "max_results" : "100",
    "tweet.fields" : "author_id",
    "user.fields" : "name"
}

def bearer_oauth(r):
    r.headers["Authorization"] = f"Bearer {bearerToken}"
    r.headers["User-Agent"] = "v2RecentSearchPython"
    return r

def connect_to_endpoint(url,params):
    response = requests.get(url,auth=bearer_oauth,params=params)#
    return response.json()

json_response = connect_to_endpoint(search_url, query_params)

parsedRes = json.dumps(json_response, indent=4, ensure_ascii=False)
print(parsedRes)

{
    "data": [
        {
            "text": "Ronnie O'Sullivan wordt door velen als de \"Greatest Of All Time\" beschouwd en behaalde in 2022 zijn zevende wereldtitel, waarmee hij op gelijke hoogte als Stephen Hendry kwam. Toch mocht ook dat niet baten, want zowel in 2020 als in 2022 landde hij buiten de top 3.",
            "author_id": "2256824641",
            "id": "1653736033521917953",
            "edit_history_tweet_ids": [
                "1653736033521917953"
            ]
        },
        {
            "text": "Die eerste plaats was voor Steve Davis in 1988. De laatste keer dat snooker in de top 3 is geëindigd dateert van 1990: een tweede plaats voor Stephen Hendry. In 2020 en 2022 was Ronnie O'Sullivan genomineerd.",
            "author_id": "2256824641",
            "id": "1653736028430057473",
            "edit_history_tweet_ids": [
                "1653736028430057473"
            ]
        },
        {
            "text": "@sporza Hij staat op nummer 2 in de wereld. 

Lifted this code from my CA2 in first semester. Based on code in one of Sam's class. 100 tweets is the max I can get. Downloading from archive.org also not doable due to the sheer volume of it all. 

In [53]:
data = pd.read_csv("elonmusk_tweets.csv")

In [54]:
data.head()

,id,created_at,text
0,849636868052275200,2017-04-05 14:56:29,b'And so the robots spared humanity ... https:...
1,848988730585096192,2017-04-03 20:01:01,"b""@ForIn2020 @waltmossberg @mims @defcon_5 Exa..."
2,848943072423497728,2017-04-03 16:59:35,"b'@waltmossberg @mims @defcon_5 Et tu, Walt?'"
3,848935705057280001,2017-04-03 16:30:19,b'Stormy weather in Shortville ...'
4,848416049573658624,2017-04-02 06:05:23,"b""@DaveLeeBBC @verge Coal is dying due to nat ..."


In [55]:
data = data.drop("id", axis=1)

In [56]:
data.head()

,created_at,text
0,2017-04-05 14:56:29,b'And so the robots spared humanity ... https:...
1,2017-04-03 20:01:01,"b""@ForIn2020 @waltmossberg @mims @defcon_5 Exa..."
2,2017-04-03 16:59:35,"b'@waltmossberg @mims @defcon_5 Et tu, Walt?'"
3,2017-04-03 16:30:19,b'Stormy weather in Shortville ...'
4,2017-04-02 06:05:23,"b""@DaveLeeBBC @verge Coal is dying due to nat ..."


In [57]:
data["created_at"]

0       2017-04-05 14:56:29
1       2017-04-03 20:01:01
2       2017-04-03 16:59:35
3       2017-04-03 16:30:19
4       2017-04-02 06:05:23
               ...         
2814    2011-12-03 08:22:07
2815    2011-12-03 08:20:28
2816    2011-12-01 10:29:04
2817    2011-12-01 09:55:11
2818    2010-06-04 18:31:57
Name: created_at, Length: 2819, dtype: object

In [58]:
data.dtypes

created_at    object
text          object
dtype: object

In [59]:
data=data.rename(columns={"created_at":"date"})
data["date"]=pd.to_datetime(data["date"],format="%Y/%m/%d")

In [60]:
data.dtypes

date    datetime64[ns]
text            object
dtype: object

In [61]:
data.date[0].year

2017

In [62]:
if data.date[0].year == 2017:
    print("Yes")
else:
    print("No")

Yes


In [63]:
len(data.index)

2819

In [64]:
x = 0;
for y in data.date:
    if y.year == 2017 or y.year == 2010:
        data.drop(x, axis=0, inplace=True)
        x+=1
    else:
        x+=1

In [65]:
data.head()

,date,text
377,2016-12-31 21:30:05,b'HW2 Autopilot software uploading to 1000 car...
378,2016-12-31 02:47:28,"b'@vicentes @DragTimes Late Jan, along with Li..."
379,2016-12-31 02:40:28,"b'@DragTimes Yes, but held up by Autopilot . I..."
380,2016-12-31 02:34:09,b'Resolving an Autopilot HW2 bug that shows up...
381,2016-12-30 19:44:02,b'Churchill (non) quotes \nhttps://t.co/avA4YD...


In [66]:
data.tail()

,date,text
2813,2011-12-04 03:33:52,b'Am reading a great biography of Ben Franklin...
2814,2011-12-03 08:22:07,b'That was a total non sequitur btw'
2815,2011-12-03 08:20:28,"b'Great Voltaire quote, arguably better than T..."
2816,2011-12-01 10:29:04,b'I made the volume on the Model S http://t.co...
2817,2011-12-01 09:55:11,"b""Went to Iceland on Sat to ride bumper cars o..."


In [67]:
data = data.reset_index()

In [69]:
data.drop("index",axis=1,inplace=True)

In [70]:
data.head()

,date,text
0,2016-12-31 21:30:05,b'HW2 Autopilot software uploading to 1000 car...
1,2016-12-31 02:47:28,"b'@vicentes @DragTimes Late Jan, along with Li..."
2,2016-12-31 02:40:28,"b'@DragTimes Yes, but held up by Autopilot . I..."
3,2016-12-31 02:34:09,b'Resolving an Autopilot HW2 bug that shows up...
4,2016-12-30 19:44:02,b'Churchill (non) quotes \nhttps://t.co/avA4YD...


In [76]:
a = 0;
b = 0;
c = 0;
d = 0;
e = 0;
f = 0;
for y in data.date:
    if y.year == 2011:
        a +=1
    if y.year == 2012:
        b += 1
    if y.year == 2013:
        c += 1
    if y.year == 2014:
        d += 1
    if y.year == 2015:
        e += 1
    if y.year == 2016:
        f += 1       
print(a,b,c,d,e,f)

44 316 478 232 436 935


In [80]:
x = 0;
for y in data.date:
    if y.year == 2016:
        x+=1
    else:
        data.drop(x, axis=0, inplace=True)
        x+=1

In [81]:
data.head()

,date,text
0,2016-12-31 21:30:05,b'HW2 Autopilot software uploading to 1000 car...
1,2016-12-31 02:47:28,"b'@vicentes @DragTimes Late Jan, along with Li..."
2,2016-12-31 02:40:28,"b'@DragTimes Yes, but held up by Autopilot . I..."
3,2016-12-31 02:34:09,b'Resolving an Autopilot HW2 bug that shows up...
4,2016-12-30 19:44:02,b'Churchill (non) quotes \nhttps://t.co/avA4YD...


In [82]:
data.tail()

,date,text
930,2016-01-10 20:11:38,"b""In ~2 years, summon should work anywhere con..."
931,2016-01-10 20:08:28,b'Tap your phone or key and your car will open...
932,2016-01-10 20:04:43,b'First baby step in Tesla Summon capability n...
933,2016-01-02 17:07:03,"b'Worth reading The Machine Stops, an old stor..."
934,2016-01-01 00:19:43,b'Falcon 9 back in the hangar at Cape Canavera...


In [83]:
data.to_csv('2016tweets.csv')